# Ephemerides calculation routine

In [1]:
from montu import *
import matplotlib.pyplot as plt
from astropy.time import Time
from astroquery.jplhorizons import Horizons

%load_ext autoreload
%autoreload 2

# Load SPICE data
Montu.load_kernels()

Loading kernel latest_leapseconds.tls
Loading kernel de441_part-1.bsp
Loading kernel de441_part-2.bsp
Loading local kernel frame.tk
Loading local kernel pck00011.tpc
Loading local kernel earth_assoc_itrf93.tf


## Final class design

In [59]:
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, FK5, ICRS
import astropy.units as u
from astropy.coordinates.erfa_astrom import erfa_astrom, ErfaAstromInterpolator
from functools import lru_cache

AU = 149597870.700 # km
CSPEED = 299792.458 # km/s 

class PlanetaryBody(object):
    """Create a planetary body

    Examples: 
        earth = PlanetaryBody(id='399')
        print(Earth.f)
    """

    @lru_cache()
    def __new__(cls,id):
        """This method is intended to avoid creating a new object with the same id
        Instead this method create a clone of the previously created object.
        """
        return super().__new__(cls)

    def __init__(self,id):

        # Obtain properties of object
        self.id = id
        n,rs=spy.bodvrd(self.id,"RADII",3)
        self.Re=rs[0]
        self.Rt=rs[1]
        self.Rp=rs[2]
        self.f=(self.Re-self.Rp)/self.Re

        # Set dummy ephemerides
        self.epochs = []
        self.eq_J2000 = SkyCoord(ra=0*u.deg,dec=0*u.deg)
        self.eq_epoch = SkyCoord(ra=0*u.deg,dec=0*u.deg)
        self.az_epoch = None

    def calculate_ephemerides(self,location,epochs,verbose=True):

        self.epochs = epochs

        # Ephemeris time
        et = epochs.et

        # Retrieve position of planet
        try:
            Montu.vprint(verbose,f"Retrieving position for object {location.planet.id}")
            planetSSBJ2000,lt = spy.spkezr(location.planet.id,et,'J2000','None','SSB')
        except:
            Montu.vprint(verbose,f"\tCorrecting id to {location.planet.id[0]}")
            planetSSBJ2000,lt = spy.spkezr(location.planet.id[0],et,'J2000','None','SSB')
            
        try:
            Montu.vprint(verbose,f"Retrieving position for object {self.id}")
            bodySSBJ2000,lt = spy.spkezr(self.id,et,'J2000','None','SSB')
        except:
            Montu.vprint(verbose,f"\tCorrecting id to {self.id[0]}")
            bodySSBJ2000,lt = spy.spkezr(self.id[0],et,'J2000','None','SSB')

        """
        This has been tested using Horizons @ 2000-01-01 12:00:00.0 UTC and it gives the 
        position exactly 
        """
        M_J2000_ECLIPJ2000 = spy.pxform('J2000','ECLIPJ2000',et)
        Montu.vprint(verbose,epochs.jd)
        Montu.vprint(verbose,f"Planet {location.planet.id} position w.r.t. SSB = ",
                     spy.mxv(M_J2000_ECLIPJ2000,planetSSBJ2000[:3]),
                     spy.mxv(M_J2000_ECLIPJ2000,planetSSBJ2000[3:])
                     )
        Montu.vprint(verbose,f"Body {self.id} position w.r.t. SSB = ",
                     spy.mxv(M_J2000_ECLIPJ2000,bodySSBJ2000[:3]),
                     spy.mxv(M_J2000_ECLIPJ2000,bodySSBJ2000[3:]),
                     )

        # Position of the observing site with respect to SSB
        location.SSBJ2000 = planetSSBJ2000[:3] + location.J2000
        
        # Celestial Coordinates at J2000
        bodyTOPOJ2000 = bodySSBJ2000[:3] - location.SSBJ2000
        r,RAbodyJ2000,DECbodyJ2000 = spy.recrad(bodyTOPOJ2000)
        self.eq_J2000 = SkyCoord(ra=RAbodyJ2000*u.rad,dec=DECbodyJ2000*u.rad)
        Montu.vprint(verbose,"Coordinates @ J2000: ",self.eq_J2000.ra/15,self.eq_J2000.dec)

        # Celestial Coordinates at Epoch
        if location.planet.id == '399':
            # Method 1: using astropy
            self.eq_epoch = self.eq_J2000.transform_to(FK5(equinox=self.epochs.astrotime))
            
            """
            # There is a second method to precess the coordinates using SPICE
            Montu.vprint(verbose,f"Coordinates @ Epoch using astropy : ",self.eq_epoch.ra/15,self.eq_epoch.dec)
            # Method 2: using SPICE
            M_J2000_Epoch = spy.pxform('J2000','EARTHTRUEEPOCH',et)
            bodyTOPOEpoch = spy.mxv(M_J2000_Epoch,bodyTOPOJ2000)
            r,RAbody,DECbody = spy.recrad(bodyTOPOEpoch)
            self.eq_epoch = SkyCoord(ra=RAbody*u.rad,dec=DECbody*u.rad)
            Montu.vprint(verbose,f"Coordinates @ Epoch using SPICE : ",self.eq_epoch.ra/15,self.eq_epoch.dec)
            """
        else:
            self.eq_epoch = SkyCoord(ra=RAbodyJ2000*u.rad,dec=DECbodyJ2000*u.rad)
        
        Montu.vprint(verbose,"Coordinates @ Epoch: ",self.eq_epoch.ra/15,self.eq_epoch.dec)

        # AltAz Coordinates at body
        self.az_epoch = self.eq_J2000.transform_to(location.frame)
        Montu.vprint(verbose,"AltAz @ Epoch: ",self.az_epoch.az.value,self.az_epoch.alt.value)      
    
    def __str__(self):
        str = f"""
Planetary body id: {self.id}
Size: {self.Re,self.Rt,self.Rp,self.f}
Epoch: {self.epochs.datestr}
Coordinates:
    equatorial J2000 = {self.eq_J2000}
    equatorial @ epochs = {self.eq_epoch}
    AltAz @ epochs = {self.az_epoch}
"""
        return str
class ObservingSite(object):
    """Create an observing site

    Attributes:
        
        location: dictionary:
            lon: float [deg]: geodetic longitude
            lat: float [deg]: geodetic latitude
            elevation: float [km]: eleveation
        
        body: dictionary:
            id: SPICE id of body
            frameid: Rotating reference frame
    """
    def __init__(self,
                 location=dict(lon=0*u.deg,lat=0*u.deg,height=0),
                 conditions=dict(pressure=0*u.hPa,temperature=0*u.deg_C,relative_humidity=0,obswl=1*u.micron),
                 onplanet=PlanetaryBody(id='399')):

        # Body
        self.planet = onplanet
        
        # Geodetic coordinates
        self.location = location  

        # Atmospheric conditions
        self.conditions = conditions 

        # Cartesian coordinates
        #kwargs = location | conditions
        kwargs = location | dict()
        self.site = EarthLocation(**kwargs)
        self.ITRF = np.array(list(self.site.value))

        # Update site with a dummy time (J2000.0)
        self.update_site(MonTime(0,format='et',scale='tt'))

    def update_site(self,mtime):  
        self.mtime = mtime
        et = mtime.et
        self.M_ITRF_J2000 = spy.pxform('IAU_EARTH','J2000',et)
        self.J2000 = spy.mxv(self.M_ITRF_J2000,np.array(list(self.site.value)))
        self.frame = AltAz(obstime=mtime.astrotime,location=self.site)

    def __str__(self):
        str = f"""
Geodetic coordinates: {self.location}
Atmospheric conditions: {self.conditions}
Cartesian coordinates:
  ITRF = {self.ITRF}
  J2000 = {self.J2000}   
Planet: {self.planet}
Present epoch: {self.mtime.datestr}
Frame: {self.frame}
"""
        return str

Let's test it:

In [87]:
# Examples
earth = PlanetaryBody(id='399')
mars = PlanetaryBody(id='499')
sun = PlanetaryBody(id='10')
moon = PlanetaryBody(id='301')

# Time of observations
mtime = MonTime('-1600-01-01 12:00:00.00',scale='utc')
print("Date = ",mtime.datestr)
print("Time = ",mtime.jd)

# Observing location
medellin = ObservingSite(location=dict(lon=-75*u.deg,lat=6*u.deg,height=0*u.km),
                         onplanet = earth)
medellin.update_site(mtime)

# Calculate ephemerides
mars.calculate_ephemerides(location=medellin,epochs=mtime,verbose=1)

Date =  -1600-01-01 12:00:00.00
Time =  1136672.0004766833
Retrieving position for object 399
Retrieving position for object 499
	Correcting id to 4
1136672.0004766833
Planet 399 position w.r.t. SSB =  [-1.30262696e+08  7.15581325e+07  6.59757711e+05] [-14.79888945 -26.17780062  -0.21212022]
Body 499 position w.r.t. SSB =  [ 2.11465772e+08  2.26211055e+07 -6.31614150e+06] [-1.08797027 25.76840812  0.51867821]
Coordinates @ J2000:  23d31m53.71185342s -4d17m45.98937256s
Coordinates @ Epoch:  20d18m11.20595914s -21d07m31.41105532s
AltAz @ Epoch:  110.40604113826734 -11.129366925990883


Conclusions:

- Computing precession and nutation is a true pain in the ass.
- Different sources give you different values: Stellarium, AstroPy, SPICE, HORIZONS.
- Supposedly, the most reliable source is HORIZONS, but ¿how can we know it?
- I have performed many tests with different times before and the precision oscilates between arcsecs and several degrees (for the most remote times) Terrible!
- Some sources of interest that should be consulted are:
    - Circular 179 of the USNO: https://aa.usno.navy.mil/downloads/Circular_179.pdf
    - Documentation for coordinates system for AstroPy: https://docs.astropy.org/en/stable/coordinates/
- The models in Stellarium pretty agrees with Horizons, meaning they are using similar sources.
- In order to make just comparisons between Horizons and MontuPython you need to:
    - Change the date format in Horizons to Gregorian (not mixed).
    - Pass the date as Julian days in UT.
- PyPlanets (which uses the analytical model VSOP87) produce results much more similar to MontuPython than NASA Horizons.  ¿Where do the differences come from? ¿is the precession and nutation models in PyPlanets more simple than those in Horizons and Stellarium.
- Stellarium uses the VSOP87 model.

In [26]:
mtime.jd

2451545.0

In [65]:
year = '1601bc'
query = Horizons(id='4',location='@0',epochs=dict(start='1601-01-01 12:00:00',stop='1601-01-01 12:01:00',step='1d'))
data = query.vectors().to_pandas()
data[['x','y','z']] *= AU
data[['vx','vy','vz']] *= AU/DAY
data[['x','y','z']],data[['vx','vy','vz']]

(              x             y             z
 0  1.128810e+08 -1.771285e+08 -6.559353e+06,
           vx        vy        vz
 0  21.470874  14.98432 -0.234728)

In [80]:
year = '1602bc'
query = Horizons(id='4',location=dict(lon=285,lat=6,elevation=0),epochs=dict(start=f'{year}-12-18 12:00:00',stop=f'{year}-12-18 12:01:00',step='1d'))
query.ephemerides()

targetname,datetime_str,datetime_jd,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,surfbright,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat
---,---,d,---,---,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,---,---,mag,mag,mag / arcsec2,%,arcsec,arcsec,---,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,---,deg,deg,%,deg,deg,deg,deg,---,s,deg,deg,deg,deg,deg,deg,---,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,---,deg,deg,deg
str19,str18,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,float64,str1,int64,int64,int64,int64,int64,str3,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64
Mars Barycenter (4),b1602-Dec-18 12:00,1136644.0,*,,333.12656,-12.63994,281.69939,-24.63237,116.7309,9.666536,113.939623,-15.603199,-7.25,818.36,97784.851,-6376.75,99.721,11.8174040122,999,--,--,--,--,--,97161.67,*,--,--,--,--,--,n.a,--,--,--,349.1325,-1.9912,1.40128321223,1.0648754,2.20553888780212,6.2423618,18.3429076,26.152097,52.4178577,26.9894,/T,18.5956,4.929,7.4097,134.4151,71.143,248.976,-0.66828,Aqr,36977.068139,280.6240592,-1.1887801,--,--,46.223853,-50.137444,7.0026418089,0.000355,--,--,--,--,--,--,--,--,--,--,--,--,29.2355,-6.962555612,18.5904,339.8306,-1.744


In [90]:
query = Horizons(id='4',location=dict(lon=285,lat=6,elevation=0),epochs=1136672)
query.ephemerides()

targetname,datetime_str,datetime_jd,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,surfbright,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat
---,---,d,---,---,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,---,---,mag,mag,mag / arcsec2,%,arcsec,arcsec,---,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,---,deg,deg,%,deg,deg,deg,deg,---,s,deg,deg,deg,deg,deg,deg,---,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,---,deg,deg,deg
str19,str25,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,float64,str1,int64,int64,int64,int64,int64,str3,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64
Mars Barycenter (4),b1601-Jan-15 12:00:00.000,1136672.0,*,,353.2685,-4.16406,304.88068,-21.04867,111.582,27.09602,110.314014,-11.481637,29.9,839.86,72200.801,8223.867,87.802,13.6573015453,999,--,--,--,--,--,73199.79,*,--,--,--,--,--,n.a,--,--,--,6.1715,-1.6949,1.42281243148,1.5756192,2.30966103046323,5.7567233,19.20886505,25.7833581,53.7382914,20.3333,/T,14.0044,8.785,1.0888,145.6623,69.109,245.761,-0.8024,Aqr,36975.416044,302.2627446,-1.0574999,--,--,80.333257,-60.38354,6.7880607783,0.000355,--,--,--,--,--,--,--,--,--,--,--,--,46.2655,-6.668077115,13.999,359.1693,-1.4346


In [103]:
mtime = MonTime('-1600-01-01 12:00:00',scale='tt')
jds = np.arange(mtime.jd,mtime.jd-1000,-100)
len(jds)

10

In [104]:
query = Horizons(id='4',location=dict(lon=285,lat=6,elevation=0),epochs=jds)
query.ephemerides()

targetname,datetime_str,datetime_jd,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,surfbright,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat
---,---,d,---,---,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,---,---,mag,mag,mag / arcsec2,%,arcsec,arcsec,---,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,---,deg,deg,%,deg,deg,deg,deg,---,s,deg,deg,deg,deg,deg,deg,---,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,---,deg,deg,deg
str19,str25,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,str1,int64,int64,int64,int64,int64,str3,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64
Mars Barycenter (4),b1604-Jul-29 12:00:00.000,1135772.0,*,m,205.93988,-10.57471,159.79838,9.07263,94.17443,-40.5547,75.819114,-30.282665,192.66,868.49,156264.206,-47215.13,98.592,2.518395673,999.0,--,--,--,--,--,168474.2,*,--,--,--,--,--,n.a,--,--,--,237.2681,0.0794,1.493937655915,-2.1656066,1.99026328431593,6.9318267,16.552515,24.6046343,41.6930716,46.7984,/T,29.3942,152.9,97.0933,103.8074,111.069,293.416,0.95219,Vir,37028.537599,157.9351772,0.3504839,--,--,323.27338,50.221206,6.9903732727,0.000355,--,--,--,--,--,--,--,--,--,--,--,--,277.3853,-8.134829359,29.3902,222.5683,0.1202
Mars Barycenter (4),b1604-Nov-06 12:00:00.000,1135872.0,*,m,282.76551,-24.21008,228.9494,-19.30453,112.612,-32.6389,108.981326,-4.432386,71.87,846.8,70013.551,-25804.71,113.046,9.0893542594,999.0,--,--,--,--,--,73530.53,*,--,--,--,--,--,n.a,--,--,--,295.8377,-1.7623,1.39629605588,-0.8889163,2.27407865044865,2.7541119,18.91293542,26.2382819,53.7862847,20.4251,/T,14.2118,37.4,2.2922,145.3631,87.062,266.483,-0.13863,Sgr,37022.633318,231.6939041,-0.8086887,--,--,11.082637,-10.694917,7.1522492946,0.000355,--,--,--,--,--,--,--,--,--,--,--,--,335.9829,-6.173939164,14.2063,288.7302,-1.5352
Mars Barycenter (4),b1603-Feb-14 12:00:00.000,1135972.0,*,,0.37828,-1.02383,312.84993,-19.08646,109.8899,31.72337,110.516199,9.330195,143.25,838.88,-6856.27,-5962.62,228.547,15.6603952726,5.901,1.659,--,--,--,--,9062.073,*,--,--,--,--,--,n.a,--,--,--,358.3276,-1.8517,1.411624220352,1.3586749,2.40984271295893,1.0078682,20.0420507,25.9742476,55.5388603,2.5172,/L,1.782,123.0,75.5754,175.7007,221.235,245.542,-0.73465,Psc,37016.729508,310.0011986,-1.0589935,--,--,96.19849,-61.278274,6.6706547714,0.000355,--,--,--,--,--,--,--,--,--,--,--,--,38.459,-5.196266568,1.7872,359.1313,-1.471
Mars Barycenter (4),b1603-May-25 12:00:00.000,1136072.0,*,m,70.66347,22.34989,20.63484,8.49348,92.86476,40.0186,83.97108,43.172113,3.63,891.01,-86902.08,-32991.48,252.872,22.231346752,1.459,0.41,--,--,--,--,94623.63,*,--,--,--,--,--,n.a,--,--,--,54.9594,-0.1645,1.523517885675,2.1757381,2.36796052417533,-4.1485752,19.69372716,24.1316424,48.4188642,26.2843,/L,17.2093,81.9,65.7178,136.5063,261.554,260.217,0.36462,Tau,37010.82617,22.2411876,-0.3476253,--,--,177.19384,-15.382234,7.1668223723,0.

Guide to Stellarium: https://stellarium.org/files/guide.pdf

## PyPlanets

Another package that provides positions with relatively good accuracy is `PyPlanets` (https://pypi.org/project/pyplanets/, https://github.com/martin5f/pyplanets). This package uses the VSOP87 semianalytic theory for computing planetary positions.

In [158]:
from pyplanets.core.epoch import Epoch
from pyplanets.core.constellation import Constellation
from pyplanets.planets.earth import Earth
from pyplanets.planets.mars import Mars

In [159]:
e0 = pyplanets.core.coordinates.mean_obliquity(-1500, 4, 10)
float(e0)


23.87061494774927

Interestingly, the `Epoch` class of PyPlanets use a mixed calendar: Gregorian after 1582-Oct-15 and Julian before.

Let's compute a position:

In [170]:
epoch = Epoch(-3000, 1, 1, 12, 0, 0)
print("Julian day = ",epoch.jde)
mars = Mars(epoch)
earth = Earth(epoch)
constellation = Constellation(earth, mars)
ra, dec, elon = constellation.geocentric_position()
Montu.dec2hex(float(ra)/15), Montu.dec2hex(float(dec))

Julian day =  <bound method Epoch.jde of Epoch(625308.0)>
[2023-10-01 15:56:32] DEBUG:pyplanets.core.constellation:Iteration-# / delta = 1 / 1.5201502632067154
[2023-10-01 15:56:32] DEBUG:pyplanets.core.constellation:Iteration-# / delta = 2 / 1.5203018255369887
[2023-10-01 15:56:32] DEBUG:pyplanets.core.constellation:Iteration-# / delta = 3 / 1.5203018189391821
[2023-10-01 15:56:32] DEBUG:pyplanets.core.constellation:Iteration-# / delta = 4 / 1.5203018189391821
[2023-10-01 15:56:32] DEBUG:pyplanets.core.constellation:Total iterations / tau[min] = 4 / 12.643964606601408


((12.0, 23, 49.36779810826664), (-1.0, 13, 34.98439819692443))

In [172]:
pyplanets.core.coordinates.mean_obliquity(epoch)

Angle(24.02100913228953)

Compare with Montu calculations:

In [179]:
# Examples
earth = PlanetaryBody(id='399')
mars = PlanetaryBody(id='499')
sun = PlanetaryBody(id='10')
moon = PlanetaryBody(id='301')

# Time of observations
mtime = MonTime(epoch.jde(),format='jd',scale='tt')
print("Date = ",mtime.datestr)
print("Time = ",mtime.jd)

# Observing location
medellin = ObservingSite(location=dict(lon=-75*u.deg,lat=6*u.deg,height=0*u.km),
                         onplanet = earth)
medellin.update_site(mtime)

# Calculate ephemerides
mars.calculate_ephemerides(location=medellin,epochs=mtime,verbose=1)

Date =  -3001-12-07 12:00:00.000
Time =  625308.0
Retrieving position for object 399
Retrieving position for object 499
	Correcting id to 4
625308.0
Planet 399 position w.r.t. SSB =  [-1.22086148e+08  8.20096617e+07  1.09791947e+06] [-17.02801759 -24.76619828  -0.26337573]
Body 499 position w.r.t. SSB =  [-1.80694180e+08 -1.37734547e+08  3.78429226e+06] [ 16.16053341 -17.82173141  -0.86656891]
Coordinates @ J2000:  16d55m29.25157321s -21d55m49.8804411s
Coordinates @ Epoch:  12d23m50.11531786s -1d13m44.06302408s
AltAz @ Epoch:  144.0832274094903 81.09466373451114


Which pretty agrees.

Interestingly, the values computed by `PyPlanet` are not close to that in Stellarium, that for this time are: RA/Dec (J2000.0): 16h57m45.23s/-21 ${ }^{\circ} 59^{\prime} 43.2^{\prime \prime}$ RA/Déc (on date): $12 \mathrm{~h} 25 \mathrm{~m} 47.75 \mathrm{~s} / \mathrm{-1}^{\circ} 27^{\prime} 02.7^{\prime \prime}$

In [165]:
import pyplanets.core.coordinates
from pyplanets.core.angle import Angle
from pyplanets.core.epoch import Epoch, JDE2000

In [169]:
# Location in the planet
lon = Angle(77, 3, 56)
lat = Angle(38, 55, 17)

# ra and declination
ra = Angle(23, 9, 16.641, ra=True)
dec = Angle(-6, 43, 11.61)

# Sidereal time
theta0 = Angle(8, 34, 57.0896, ra=True)

# Obliquity
eps = Angle(23, 26, 36.87)

# Compute correction to convert from mean to apparent sidereal time
delta = Angle(0, 0, ((-3.868 * np.cos(eps.rad())) / 15.0), ra=True)
theta0 += delta

# Hour angle
h = theta0 - lon - ra

# Conversion
azi, ele = pyplanets.core.coordinates.equatorial2horizontal(h, dec, lat)
print("Equatorial to horizontal: Azimuth", round(azi, 3))  # 68.034
print("Equatorial to horizontal: Elevation", round(ele, 3))  # 15.125

Equatorial to horizontal: Azimuth 68.034
Equatorial to horizontal: Elevation 15.125


## PyEphem

Manual: 
Comment on theory used by PyEphem: https://stackoverflow.com/a/27985894 

In [173]:
!pip install pyephem

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyephem: filename=pyephem-9.99-py3-none-any.whl size=1568 sha256=c722cc209f27fc4c0080c7b826d1214432f68a287d85aa100092ea867d60371c
  Stored in directory: /Users/jorgezuluagacallejas/Library/Caches/pip/wheels/02/73/42/2da2a6ffa002ca833e62efa578a65914eba0c5f3a188c07c02
Successfully built pyephem
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [174]:
import ephem

In [175]:
mars = ephem.Mars()

In [213]:
mars.compute('-3001/1/1 12:00:00')
print(mars.ra, mars.dec)

12:25:53.30 -1:27:41.3


In [214]:
boston = ephem.Observer()
boston.lat = '6'
boston.lon = '-75'
boston.elevation = 0
boston.temp = 0
boston.pressure = 0
boston.date = '-3001/1/1 12:00:00'
mars = ephem.Mars()
mars.compute(boston)
print(float(mars.az)*RAD, float(mars.alt)*RAD)

142.35529445502598 80.59871839500835


In [203]:
ephem.Observer?

Init signature: ephem.Observer(self, /, *args, **kwargs)
Docstring:     
A location on earth for which positions are to be computed.

An `Observer` instance allows you to compute the positions of
celestial bodies as seen from a particular latitude and longitude on
the Earth's surface.  The constructor takes no parameters; instead,
set its attributes once you have created it.  Defaults:

`date` - the moment the `Observer` is created
`lat` - zero latitude
`lon` - zero longitude
`elevation` - 0 meters above sea level
`horizon` - 0 degrees
`epoch` - J2000
`temp` - 15 degrees Celsius
`pressure` - 1010 mBar
File:           ~/opt/anaconda3/lib/python3.9/site-packages/ephem/__init__.py
Type:           type
Subclasses:     


This is definitively the best package.

## SkyField

In [187]:
!pip install skyfield

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [215]:
from skyfield.api import load

# Create a timescale and ask the current time.
ts = load.timescale()
t = ts.tt_jd(mtime.jd)

# Load the JPL ephemeris DE421 (covers 1900-2050).
planets = load('de441.bsp')
earth, mars = planets['earth'], planets['mars barycenter']

"""
# What's the position of Mars, viewed from Earth?
astrometric = earth.at(t).observe(mars)
ra, dec, distance = astrometric.radec()

print(ra)
print(dec)
print(distance)
"""

[                                 ]   2% de441.bsp

KeyboardInterrupt: 

In [218]:
!ls

TODO.md                       test-basic-tools.ipynb
data                          test-ephemerides.ipynb
gallery                       test.py
montu-mars-conjunctions.ipynb tmp
montunctions.ipynb


In [217]:
!rm -rf de441*